# Enviroment set up

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
FILE_PATH = "/gdrive/My Drive/Colab Notebooks/Master/Vision por Computador/Practica 1/" #Guillermo
#FILE_PATH = "/gdrive/MyDrive/Colab Notebooks/COM VIS/Practica 1/" #Antonio

# Imports

In [5]:
import glob

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import pickle


from tensorflow.keras.layers import Input, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, SeparableConv2D, MaxPooling2D, add, UpSampling2D, Cropping2D
from tensorflow.keras import Model, losses
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from keras.layers.merge import concatenate
from keras.initializers import GlorotNormal
from keras.models import load_model

from scipy.spatial.distance import euclidean

from sklearn.cluster import KMeans

# Data load

In [6]:
with open(FILE_PATH + 'binary_60x80.pkl', 'rb') as file:
    X = pickle.load(file)

KeyboardInterrupt: ignored

In [ ]:
X.shape

# Architecture definition
Code base on [Image segmentation with a U-Net-like architecture](https://keras.io/examples/vision/oxford_pets_image_segmentation/)

In [ ]:
def get_model():
    inputs = Input(shape=(80, 60, 3))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Generate a vector with the representation of the image
    x = Flatten()(x)

    x = Dense(10)(x)
    x = Activation("sigmoid")(x)

    encoder = Model(inputs, x)

    x = Dense(5*4*256)(x)
    x = Activation("relu")(x)

    x = Reshape((5, 4, 256))(x)

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        if filters != 256:
            # Project residual
            residual = UpSampling2D(2)(previous_block_activation)
            residual = Conv2D(filters, 1, padding="same")(residual)
            x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = Conv2D(3, 3, activation="sigmoid", padding="same")(x)
    outputs = Cropping2D(cropping=((0, 0), (2, 2)), data_format=None)(outputs)

    # Define the model
    autoencoder = Model(inputs, outputs)
    return encoder, autoencoder

In [ ]:
encoder, autoencoder =  get_model()

## Architecture plot

In [ ]:
plot_model(encoder, show_layer_names=True, show_shapes=True)

In [ ]:
plot_model(autoencoder, show_layer_names=True, show_shapes=True)

## Plotting functions definition

In [ ]:
def plot_reconstruction(n_plots, reconstructed_img_predictions, epoch, plot=False):
    f, axarr = plt.subplots(n_plots, 2, figsize=(6, 12))
    f.suptitle('Epoch: ' + str(epoch), fontsize=16)

    axarr[0,0].set_title('Source image')
    axarr[0,1].set_title('Reconstructed image')


    for i in range(n_plots):
        axarr[i,0].imshow(X[i])
        axarr[i,1].imshow(reconstructed_img_predictions[i])

        axarr[i,0].get_xaxis().set_visible(False)
        axarr[i,0].get_yaxis().set_visible(False)
        axarr[i,1].get_xaxis().set_visible(False)
        axarr[i,1].get_yaxis().set_visible(False)

    if plot:
        plt.plot()
    plt.savefig(FILE_PATH + "Results/" + epoch + 'reconstruction_results.png')

In [ ]:
def plot_train_val(history):
    plt.rcParams['figure.figsize'] = [15, 5]
    f, ax1 = plt.subplots(1, 1, sharex=True)

    ax1.plot(history.history['mae'], label='mae (train)')
    ax1.plot(history.history['val_mae'], label='mae (test)')
    ax1.set_title('MAE')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    plt.plot()
    plt.savefig(FILE_PATH + "Results/losses.png")

## Model training

We define a custom Callback for saving the results while the training is running

In [ ]:
class SaveResults(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        plot_reconstruction(5, self.model.predict(X[:100]), epoch=str(epoch))

In [ ]:
with tf.device('/device:GPU:0'):
    tf.keras.backend.clear_session()
    autoencoder.compile(
        loss='mse', 
        optimizer=Adam(), metrics=['mae'])
    history = autoencoder.fit(X[:40000], X[:40000],
                        epochs=50, batch_size=64, verbose=1,
                        validation_data=(X[40000:44400], X[40000:44400]),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss",),
                                   SaveResults()])

### Save weights

In [ ]:
autoencoder.save(FILE_PATH + 'Results/autoencoder.h5')
encoder.save(FILE_PATH + 'Results/encoder.h5')

### Load weights

In [ ]:
'''autoencoder = load_model(FILE_PATH + 'Results_3dim/autoencoder.h5')
encoder = load_model(FILE_PATH + 'Results_3dim/encoder.h5')'''

## Training evaluation

In [ ]:
plot_train_val(history)

In [ ]:
plot_reconstruction(5, autoencoder.predict(X[:100]), epoch='FinalResults')

# Recommendation system

The feature descriptor of each image is generated using the encoder. This way we do a dimensionality reduction.

In [ ]:
encoder_predictions = encoder.predict(X[:1000])

In [ ]:
encoder_predictions.shape

## Image distribution in space

Muestreo de la distribución espacial de los resúmenes de cada imágen, cabe destacar que cada imagen la representan sus 2 dimensiones

In [ ]:
'''fig = go.Figure(data=[go.Scatter3d(x=encoder_predictions[:,0], y=encoder_predictions[:,1], z=encoder_predictions[:,2],
                                   mode='markers')])
fig.show()'''

## Recommendation via clustering

In [ ]:
kmeans = KMeans(n_clusters = 6, random_state=0)
kmeans.fit(encoder_predictions)

In [ ]:
clusters = kmeans.predict(encoder_predictions)

In [ ]:
'''fig = go.Figure(data=[go.Scatter3d(x=encoder_predictions[:,0], y=encoder_predictions[:,1], z=encoder_predictions[:,2],
                                   mode='markers',
                                   marker=dict(size=6,
                                               color=clusters,                # set color to an array/list of desired values
                                               colorscale='Viridis',   # choose a colorscale
                                               opacity=1))])
fig.show()'''

In [ ]:
def clusters_full(cont_cluster, n_plots):
    for cluster in cont_cluster:
        if cluster < n_plots:
            return True

    return False

def plot_reconstruction_clusters(n_plots, n_clusters, imgs, clusters, plot=False):
    f, axarr = plt.subplots(n_clusters, n_plots, figsize=(3*n_plots, 3*n_clusters))
    cont = 0
    cont_cluster = [0 for i in range(n_clusters)]

    while clusters_full(cont_cluster, n_plots):
        if cont_cluster[clusters[cont]] < n_plots:
            axarr[clusters[cont], cont_cluster[clusters[cont]]].imshow(imgs[cont])
            axarr[clusters[cont], cont_cluster[clusters[cont]]].get_xaxis().set_visible(False)
            axarr[clusters[cont], cont_cluster[clusters[cont]]].get_yaxis().set_visible(False)
            
            cont_cluster[clusters[cont]] = cont_cluster[clusters[cont]]+1
        cont = cont+1

    if plot:
        plt.plot()
    plt.savefig(FILE_PATH + 'Results/recomendation_clusters_results.png')

In [ ]:
plot_reconstruction_clusters(4, max(clusters)+1, X, clusters, plot=True)

## Neighbor list

In [ ]:
def find_nearest_neighbor_list(img_index, encoder_predictions):
    img = encoder_predictions[img_index]
    img_nneighbors = []
    distances = []
    cont = 0

    for candidate in encoder_predictions:
        if cont != img_index:
            distances.append(euclidean(img, candidate))
            img_nneighbors.append(cont)
        cont = cont+1

    return img_nneighbors, distances

In [ ]:
def plot_nearest_neighbor_list(reconstructed_img_predictions, img_index, dataset, num_neighbors, n):
    neighbor_list, distances = find_nearest_neighbor_list(img_index, reconstructed_img_predictions)

    f, axarr = plt.subplots(1, num_neighbors + 1, figsize=(3*num_neighbors, 4))

    f.suptitle('Nearest neighbors', fontsize=16)

    axarr[0].set_title('Source image')
    axarr[0].imshow(dataset[img_index])
    axarr[0].get_xaxis().set_visible(False)
    axarr[0].get_yaxis().set_visible(False)
    
    neighbors = zip(distances, neighbor_list)
    neighbors = sorted(neighbors)
    neighbors = list(neighbors)

    for i in range(num_neighbors):
        axarr[i+1].set_title('Dist: %.2f' % neighbors[i][0])
        axarr[i+1].imshow(dataset[neighbors[i][1]])
        axarr[i+1].get_xaxis().set_visible(False)
        axarr[i+1].get_yaxis().set_visible(False)

    plt.savefig(FILE_PATH + 'Results/nearest_' + str(num_neighbors) + 'neighbors_results' + n + '.png')

In [ ]:
plot_nearest_neighbor_list(encoder_predictions, 1, X[:1000], 5, 1)

In [ ]:
plot_nearest_neighbor_list(encoder_predictions, 10, X[:1000], 5, 2)

In [ ]:
plot_nearest_neighbor_list(encoder_predictions, 150, X[:1000], 5, 3)

In [ ]:
plot_nearest_neighbor_list(encoder_predictions, 103, X[:1000], 5, 4)

In [ ]:
plot_nearest_neighbor_list(encoder_predictions, 240, X[:1000], 5, 5)